<a href="https://colab.research.google.com/github/Petrichoeur/Neural_Net_from_scratch/blob/master/Pho2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
PATH ='drive/My Drive/data'
from google.colab import drive
drive.mount('/content/drive')
!nvidia-smi

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mon Apr 27 23:12:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    12W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+--------------------------------------

In [2]:
import joblib
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import numpy as np  
import tensorflow.compat.v1 as tf  
import re 
import sys
tf.disable_v2_behavior()
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
from tqdm import trange 
from tqdm import tqdm_gui


    
def ipa_(x):

  x=x.replace('ɑ̃','0')
  x=x.replace('ɔ̃','1') 
  x=x.replace('ɛ̃', '2') 
  x=x.replace('œ̃','3') 
  x=x.replace('tr','4')
  x=x.replace('kr','5')
  x=x.replace('dr','6')
  x=x.replace('gr','7') 
  x=x.replace('ʁ','r') 
  return x

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Instructions for updating:
non-resource variables are not supported in the long term


# Import data 


In [3]:


dfdic=joblib.load(PATH+'/phono/final_data.job') 
print('dfdic is original data ')

for key in dfdic.keys():
    dfdic[key]['valeur phono enfant']=dfdic[key]['valeur phono enfant'].apply(ipa_).apply(lambda x:list(x) )

dfdic is original data 


# General way of making Pho2Vec with skip gram methodology  
Inspired by this article : 
https://towardsdatascience.com/learn-word2vec-by-implementing-it-in-tensorflow-45641adaf2ac

In [0]:
class Pho2Vec():
    
    def __init__(self,data):
        self.vocab=set(np.sum(data))
        self.vocab_len= len(self.vocab) 
        self.vocab_len_input=self.vocab_len 
        self.vocab_len_output=self.vocab_len
        self.model =None
        self.data=data
        self.pho_vec={}
        self.pho2int={} 
        self.int2pho={}
        self.history={}
        self.vector_representation = None 
        for i,word in enumerate(self.vocab):
            self.pho2int[word] = i
            self.int2pho[i] = word
    
    
    def dataforw2c(self,WORD,WINDOW_SIZE):
        data = []
        for talk in WORD:
            for pho_index, pho in enumerate(talk):
                iterab =talk[max(pho_index - WINDOW_SIZE, 0) : min(pho_index + WINDOW_SIZE, len(talk)) + 1]
                for nb_pho in iterab : 
                    if nb_pho != pho:
                        data.append([pho, nb_pho]) 
        return data 
    


        
        # function to convert numbers to one hot vectors
    def __to_one_hot(self,data_point_index, vocab_len):
        temp = np.zeros(vocab_len)
        temp[data_point_index] = 1
        return temp 
    def train_test(self,train_size=0.9,window_size=2): 
        
        self.window_size=window_size
        self.data_w2=self.dataforw2c(self.data , self.window_size)
        
        self.x = [] # input word
        self.y= [] # output word
        for data_word in self.data_w2:
            self.x.append(self.__to_one_hot(self.pho2int[ data_word[0] ], self.vocab_len))
            self.y.append(self.__to_one_hot(self.pho2int[ data_word[1] ], self.vocab_len))
        # convert them to numpy arrays
        self.x= np.asarray(self.x)
        self.y= np.asarray(self.y) 
        
        self.x_train,self.x_test,self.y_train,self.y_test =train_test_split(self.x,
                                                                           self.y, 
                                                                           train_size=train_size)
        
    
    
    
    
    
    
    
    
    
    def model_2vecbow(self,embedding_dim=6,n_iter=10000): 
        try : 
            x_train = self.x_train 
            x_test  = self.x_test 

            y_train = self.y_train 
            y_test = self.y_test  
        except : 
            AttributeError('train and test set must be initialize')
        
        
        x = tf.placeholder(tf.float32, shape=(None, self.vocab_len_input))
        y_label = tf.placeholder(tf.float32, shape=(None, self.vocab_len_output))   
        
        
        self.embedding_dim=embedding_dim
        EMBEDDING_DIM = embedding_dim
        W1 = tf.Variable(tf.random_normal([self.vocab_len_input, EMBEDDING_DIM]))
        b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM]))
        

        W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, self.vocab_len_output]))
        b2 = tf.Variable(tf.random_normal([self.vocab_len_output]))
        
        def bow(val):
            hidden_representation = tf.add(tf.matmul(val,W1), b1)
            prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_representation, W2), b2)) 
            return prediction


        cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(tf.clip_by_value(bow(x),1e-10,1.0)), reduction_indices=[1]))

        sess = tf.Session()
     
       
        Adam_opti=tf.train.AdamOptimizer(learning_rate=0.001,  beta1=0.9, beta2=0.999,   epsilon=1e-08, use_locking=False,name='Adam') 
        #train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy_loss)  
        train_step=Adam_opti.minimize(cross_entropy_loss) 

        init = tf.global_variables_initializer()
        sess.run(init) 
        
       


       
        # train for n_iter iterations
        self.history['loss']=[]
        self.history['validation_loss']=[]

        for i in range(n_iter):
 
            
            _,loss = sess.run([train_step,cross_entropy_loss], feed_dict={x: x_train, y_label: y_train}) 

            val_loss= cross_entropy_loss.eval(session=sess, feed_dict={x:x_test, y_label: y_test})  
            self.history['loss'].append(loss) 
            self.history['validation_loss'].append(val_loss)
        
            if i % 10 == 0 : 

                try : 
                    sys.stdout.write(f'progression : {i} / {n_iter}  val loss={val_loss}  loss = {loss}') 
                    clear_output(wait=True)
                except : 
                    sys.stdout.write(str_loss)
                
        
            #tbar.refresh()
        def get_pho_vectors(self): 
            try :
                for key,val in self.pho2int.items():
                    self.pho2vec[key]=vectors[val] 
            except : 
                AttributeError('train model_2vec first')
            



In [0]:

def Data4spec2vec(dfdic):

    data_contain=[]
    name= {}
    idx=0
    for key,df in dfdic.items():    
        name[idx]=key
        for age in df.age.unique():  
            data_contain= data_contain + df[df.age==age]['valeur phono enfant'].apply(lambda x: [it + f'_{(idx,int(age))}' for it in x]).to_list()
        idx+=1
    return np.array(*[data_contain]),name





class Spec2vec(Pho2Vec): 
    def __init__(self,data,name):
        super().__init__(data)  
        self.name=name
        self.pho2target={}
        self.target2pho={}

    
    def __toh_all(self,X):  
        X = np.vectorize(lambda X: self.pho2int[X])(X)
        
        return  np.eye(self.vocab_len_input)[X]
    
    def __toh_target(self,X): 
        X = np.vectorize(lambda X: self.pho2target[X])(X)
        return  np.eye(self.vocab_len_output)[X]


    def train_test(self,train_size=0.9,window_size=2): 
        
        self.window_size=window_size
        self.data_w2=self.dataforw2c(self.data , self.window_size)
        
        self.x = [] # input word
        self.y= [] # output word
        for data_word in self.data_w2:
            self.x.append(data_word[0])
            self.y.append(data_word[1])
        # convert them to numpy arrays
        self.x= np.asarray(self.x)
        self.y= np.vectorize(lambda x : re.sub(r'\([^()]*\)|_', '', x))(np.asarray(self.y)) # Getting rid of specification indice   
        
        
        for i,word in enumerate(np.unique(self.y)):
            self.pho2target[word] = i
            self.target2pho[i] = word  
        self.vocab_len_output=len(self.pho2target)
        self.x =self.__toh_all(self.x)
        self.y =self.__toh_target(self.y)
        
        self.x_train,self.x_test,self.y_train,self.y_test =train_test_split(self.x,
                                                                           self.y, 
                                                                           train_size=train_size)  
        



In [0]:
#d=Spec2vec(*Data4spec2vec(dfdic)) 
data,name = Data4spec2vec(dfdic)

Children class for Colaje use 

In [0]:
d=Spec2vec(data,name)

In [0]:
d.train_test(train_size=0.95,window_size=2)

Préparation of data for good fromat to Spec2vec

In [11]:
d.x.shape

(295712, 979)

In [12]:
d.y.shape

(295712, 40)

5.623413251903491

In [0]:
d.model_2vecbow(embedding_dim=6,n_iter=10000)

progression : 4100 / 10000  val loss=2.817044496536255  loss = 2.8024048805236816